# Tópicos de Industria I
## Práctica 2 - Vuelos de Lévy

### Universidad de Guadalajara - 2023

**Nombre:** Roberto Octavio Carrillo Luevano

**e-mail:** roberto.carrillo7958@alumnos.udg.mx

## Vuelos de Lévy

Como parte de 

## MODULES

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from plotly.subplots import make_subplots
import plotly.graph_objects as go


from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

## CLASSES

In [2]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## Actividad 1: Lévy distribution - N Curves (3 pts)

* Generar al menos las 4 curvas de distribución timpo Lévy stable que se observan en la imagen
* Visualizar con **plotly**

* Parametros m = 3.0, Levy_exponent = x, beta = 0

![levy.png](attachment:levy.png)

In [ ]:
# Parámetros Iniciales

# Resolución
resolution = 200
# Definiendo vector auxiliar
aux_domain = np.linspace(0, 2*np.pi, resolution)
# Definiendo un conjunto de exponentes para generar diferentes distribuciones
CRW_exps = [0.00001, 0.3, 0.4, 0.6, 0.9]
# CRW Figure
fig_wrapcauchy_pdf = go.Figure()
# Create aux plot
aux_plot = np.linspace(-np.pi,np.pi, resolution)

# Definiendo función para generar wrapcauchy
def wrapcauchy_pdf_exp(CWR_exp):
  return np.array([wrapcauchy.pdf(i, CWR_exp) for i in aux_domain])

def plot_wrapcauchy_pdf(wrapcauchy_pdf, plotname):
  # Define concatenated plot
  plot_wc_pdf = np.concatenate((wrapcauchy_pdf[int(resolution/2):resolution], wrapcauchy_pdf[0:int(resolution/2)]), axis=0)
  # Add the trace to the figure
  fig_wrapcauchy_pdf.add_trace(go.Scatter( 
    x = aux_plot,
    y = plot_wc_pdf,
    marker = dict(size=2),
    line = dict(width=2),
    mode = 'lines',
    name = plotname,
  ))

In [ ]:
for exp in CRW_exps:
  wrapcauchy_pdf = wrapcauchy_pdf_exp(exp)
  plot_wrapcauchy_pdf(wrapcauchy_pdf, f'cauchy_{exp}')

fig_wrapcauchy_pdf.update_layout(title_text='Wrap Cauchy Density Functions at different exponents')
fig_wrapcauchy_pdf.show()

## Actividad 2: Lévy distribution - Histogram + Curve (4 pts)

* A partir de un conjunto de valores aleatorios extraidos de una distribución tipo Lévy stable, obtener su histograma y comparar con la curva correspondiente PDF
* Visualizar con **matplotlib**

## Actividad 3: Lévy ﬂight - Vec2d - 1 Trajectory (6 pts)

* Utilizar **Vec2d**
* Guardar trayectorias en **Pandas** Data Frame
* Visualizar proyección en 3d con **plotly**

## Actividad 4: Correlated Random Walk - Vec2d - N Trajectories (7 pts)

* Generar tantas trayectorias tipo CRW como se especifique en una variable n_traj
* Cada una de las debe tener un coeficiente de Cauchy diferente
* Utilizar **Vec2d**
* Guardar trayectorias en **Pandas** Data Frame
* Visualizar proyección en 3d con **plotly**
* Exportar trayectorias como *pkl*